In [0]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
sns.set_style('darkgrid')
warnings.filterwarnings('ignore')
from sklearn.preprocessing import PolynomialFeatures


In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [27]:
train.shape, test.shape

((4177, 363), (1000, 362))

In [0]:
df = train.append(test, ignore_index=True, sort=False)

In [29]:
df.head()

,id,span,target,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,...,feature_20_type5,feature_21_type5,feature_22_type5,feature_23_type5,feature_24_type5,feature_25_type5,feature_26_type5,feature_27_type5,feature_28_type5,feature_29_type5,feature_30_type5,feature_31_type5,feature_32_type5,feature_33_type5,feature_34_type5,feature_35_type5,feature_36_type5,feature_37_type5,feature_38_type5,feature_39_type5,feature_40_type5,feature_41_type5,feature_42_type5,feature_43_type5,feature_44_type5,feature_45_type5,feature_46_type5,feature_47_type5,feature_48_type5,feature_49_type5,feature_50_type5,feature_51_type5,feature_52_type5,feature_53_type5,feature_54_type5,feature_55_type5,feature_56_type5,feature_57_type5,feature_58_type5,feature_59_type5
0,M00yaXQwMVJ1R1Q0RHFBVVR5BHyNyIeEd7yqxidFW7U=,17,1.025179,1.455438,0.767232,1.103207,0.355392,0.144054,1.249518,0.301203,-1.470791,-0.934316,0.886449,-1.113141,1.049017,1.222423,0.788908,-1.541238,1.536722,-1.682130,-1.546657,-1.297385,0.333717,0.149473,0.729299,0.387906,-0.744653,1.341640,0.404163,-2.083132,-0.674207,1.157396,0.750975,0.870192,0.875611,0.593826,-0.603761,-0.294881,0.723880,0.994828,...,0.477301,0.858831,-1.725857,1.844273,-0.598755,0.930897,0.068039,-1.364287,0.886032,-0.308191,0.871548,0.697917,-0.663403,-1.169636,0.207404,-0.733880,-0.462923,-0.226763,0.305259,0.198925,0.564382,-0.416291,0.852825,-0.076624,0.528348,0.183028,-0.655101,-0.964741,-0.756489,1.559715,-1.395021,-1.222626,-0.139682,0.813436,-0.340162,-0.032112,1.740059,1.000138,-1.667038,-0.128554
1,M01HU2pzN1dZNnVOYmhjayLgjCmIG1AyHvfRsZv+IV4=,23,0.996383,0.132304,1.724691,1.517504,0.884100,-0.897716,-0.122241,1.523423,-1.152261,-0.915475,-0.714206,0.043509,-0.222875,-0.483340,-0.737885,-0.501099,0.132304,-1.625833,-1.353529,-0.962832,0.469725,1.570780,-1.371288,1.612218,-0.317589,0.629555,-0.140000,-0.613572,0.860422,1.487905,-0.613572,1.511584,-0.353107,-1.122663,-0.240634,1.103128,-0.003848,1.274798,...,0.933283,-0.898041,1.064092,-2.211074,1.575640,-1.051326,-0.259730,0.053807,0.096736,0.970818,-0.963895,-0.228488,0.802025,0.387122,0.875071,-0.756669,-0.214778,1.657901,0.502198,0.418139,-0.127123,1.193327,0.517931,0.865856,1.022512,-0.282880,-0.770379,-0.185785,-0.769705,-1.371605,-1.269116,0.477700,1.590249,0.166410,0.648515,0.128202,-1.853485,-0.185335,-0.933777,0.420162
2,M01la1Znd0ROMUU5UGJvdc5EJqBI/dj9r8C86lWkehA=,29,1.006573,0.542726,0.844240,-0.089719,0.204441,0.300044,0.564788,0.976612,0.513310,-1.008970,-0.251507,-0.222091,0.373584,-0.339755,1.300189,-0.030887,0.219149,1.226649,0.211795,0.542726,-1.722309,0.447124,0.138255,0.998674,-1.031032,0.380938,0.505956,0.322106,-2.237090,0.469186,0.101485,0.050007,0.733930,-0.663332,-0.692748,-2.795994,0.454478,0.285336,...,-0.868428,-1.540314,1.698789,-0.091653,0.744815,-2.165877,-1.396634,1.072285,-0.878596,1.271327,-1.049581,-0.435882,1.109570,1.818930,-0.502920,1.304470,1.143654,0.772685,-0.792163,-0.970303,-0.014635,0.104753,-0.570711,-1.535606,-0.234768,-1.033575,0.613941,1.133485,0.445027,-1.301538,0.551799,1.066635,0.126409,-1.462542,1.177361,-1.142417,-0.506686,-0.741696,0.565734,-1.304175
3,M052cFV0RUdYT3FRMU1Kab7l27PQanylu/gYY6UjtK0=,16,0.968543,0.253863,0.799112,-1.399473,-1.979899,0.086771,-1.214792,-0.423301,1.159680,0.632020,-0.599188,2.311739,-0.845429,-1.188409,-0.229825,1.458688,-1.522593,0.869467,0.799112,0.728758,-0.370535,0.596843,-0.933373,-0.423301,1.230035,-0.581599,0.368190,1.564220,-0.150676,-1.610537,-0.053939,-1.707275,0.500105,-0.335358,1.696135,-0.194648,-0.906990,-0.616776,...,1.394487,0.030902,-1.139577,-0.811661,-1.481612,0.080544,2.045310,-1.033460,0.249057,-0.999302,-0.2145

In [0]:
df=pd.get_dummies(df, columns=['span'],drop_first=True)

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
lb=LabelEncoder()

In [0]:
df['id']=lb.fit_transform(df.id)


In [0]:
train_df = df[df['target'].isnull()!=True]
test_df = df[df['target'].isnull()==True]
test_df.drop('target', axis=1, inplace=True)


In [0]:
train_df=train_df.fillna(0)

In [0]:
test_df=test_df.fillna(0)

In [37]:
train_df.shape, test_df.shape

((4177, 394), (1000, 393))

In [0]:
#train_df['target'] = np.log1p(train_df['target'])

In [0]:
X = train_df.drop(labels=['target'], axis=1)
y = train_df['target'].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [0]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=300, random_state=1)
rf.fit(X, y)
y_pred_rf = rf.predict(X_test)

In [0]:
from sklearn.metrics import mean_squared_log_error,mean_squared_error,mean_absolute_error
from math import sqrt

In [0]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [43]:
print('RMSE:', rmse(y_test,y_pred_rf))

RMSLE: 0.005451516347942205


In [47]:
from sklearn.ensemble import BaggingRegressor
br = BaggingRegressor( n_estimators=300, random_state=1)
br.fit(X, y)
y_pred_br = br.predict(X_test)
print('RMSE:', rmse(y_test,y_pred_br))

RMSE: 0.005457644479035296


In [48]:
!pip install catboost

     |████████████████████████████████| 63.6MB 125kB/s 


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor


In [0]:
categorical_features_indices = np.where(X_train.dtypes == 'category')[0]

In [63]:
model =  CatBoostRegressor(iterations=2000,
                             learning_rate=0.01,
                             depth=10,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 75,
                             od_wait=100)
model.fit(X, y,
                 eval_set=(X_test, y_test),
                 cat_features=categorical_features_indices,
                 use_best_model=True)

0:	learn: 0.0201278	test: 0.0206973	best: 0.0206973 (0)	total: 877ms	remaining: 29m 12s
75:	learn: 0.0176130	test: 0.0181101	best: 0.0181101 (75)	total: 1m 5s	remaining: 27m 45s
150:	learn: 0.0158078	test: 0.0162647	best: 0.0162647 (150)	total: 2m 10s	remaining: 26m 39s
225:	learn: 0.0143696	test: 0.0148132	best: 0.0148132 (225)	total: 3m 15s	remaining: 25m 34s
300:	learn: 0.0131488	test: 0.0135695	best: 0.0135695 (300)	total: 4m 20s	remaining: 24m 29s
375:	learn: 0.0120621	test: 0.0124828	best: 0.0124828 (375)	total: 5m 25s	remaining: 23m 24s
450:	learn: 0.0111257	test: 0.0115309	best: 0.0115309 (450)	total: 6m 30s	remaining: 22m 19s
525:	learn: 0.0102825	test: 0.0106888	best: 0.0106888 (525)	total: 7m 34s	remaining: 21m 15s
600:	learn: 0.0095228	test: 0.0099210	best: 0.0099210 (600)	total: 8m 39s	remaining: 20m 10s
675:	learn: 0.0088332	test: 0.0092310	best: 0.0092310 (675)	total: 9m 45s	remaining: 19m 5s
750:	learn: 0.0082474	test: 0.0086518	best: 0.0086518 (750)	total: 10m 49s	rema

In [64]:
y_pred_cat = model.predict(X_test)
print('RMSE:', rmse(y_test,y_pred_cat))

RMSE: 0.003167384101263437


In [65]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(loss='ls', 
                               learning_rate=0.3, 
                               n_estimators=300, 
                               subsample=1.0, 
                               criterion='friedman_mse', 
                               min_samples_split=30, 
                               min_samples_leaf=1, 
                               min_weight_fraction_leaf=0.0, 
                               max_depth=7, 
                               min_impurity_decrease=0.0, 
                               min_impurity_split=None, 
                               init=None, 
                               random_state=0, 
                               max_features=None, 
                               alpha=0.9, 
                               verbose=100, 
                               max_leaf_nodes=None, 
                               warm_start=False, 
                               presort='auto')
gb.fit(X, y)
y_pred4 = gb.predict(X_test)
print('RMSE:', rmse(y_test,y_pred4))


      Iter       Train Loss   Remaining Time 
         1           0.0004            1.68m
         2           0.0003            1.87m
         3           0.0002            1.87m
         4           0.0002            1.75m
         5           0.0002            1.77m
         6           0.0002            1.79m
         7           0.0002            1.75m
         8           0.0001            1.74m
         9           0.0001            1.72m
        10           0.0001            1.67m
        11           0.0001            1.66m
        12           0.0001            1.64m
        13           0.0001            1.61m
        14           0.0001            1.60m
        15           0.0001            1.56m
        16           0.0001            1.54m
        17           0.0001            1.52m
        18           0.0001            1.50m
        19           0.0001            1.47m
        20           0.0001            1.46m
        21           0.0001            1.47m
        2

In [70]:
import lightgbm as lgb
train_data = lgb.Dataset(X, label=y)
test_data = lgb.Dataset(X_test, label=y_test)

param = {'objective': 'regression',
         'boosting': 'gbdt',
         'num_iterations': 3000,   
         'learning_rate': 0.06,  
         'num_leaves': 40,  
         'max_depth': 24,   
         'min_data_in_leaf':11,  
         'max_bin': 4, 
         'metric': 'l2_root'
         }

lgbm = lgb.train(params=param,
                 verbose_eval=1000,
                 train_set=train_data,
                 valid_sets=[test_data])

y_pred2 = lgbm.predict(X_test)
print('RMSE:', rmse(y_test,y_pred2))


[1000]	valid_0's rmse: 0.000353401
[2000]	valid_0's rmse: 2.3564e-05
[3000]	valid_0's rmse: 1.6616e-06
RMSE: 1.6618369485968043e-06


In [87]:
y_pred = y_pred_rf*0.00 + y_pred_br*0.00 + y_pred2*0.50 + y_pred_cat*0.00 + y_pred4*0.50
print('RMSE:', rmse(y_test,y_pred))



RMSE: 0.00016201435652049266


In [79]:
y_pred

array([1.00045554, 1.00652168, 0.97346712, ..., 0.96348248, 0.99427722,
       1.04142504])